## Create PostgreSQL Table for Stack Annoations

In [74]:
import os, sys, csv
import psycopg2

#### Obtain data from Zbr Annotations spreadsheet

In [75]:
file_path = os.path.dirname(os.path.realpath('__file__')) + "/ZBr Annotations.csv"

In [76]:
with open(file_path) as zbr_csv:
    
    csvreader = list(csv.reader(zbr_csv, dialect=csv.excel_tab,
                    delimiter=',', quotechar='"'))
    table = csvreader[1:]
    headers = csvreader[0]
    csv_dict = dict()
    
    for i, row in enumerate(table):
        if not row[0]:
            continue
        current_row = csv_dict[row[0]] = dict()
        for j, col in enumerate(row):
            if headers[j] == 'Keywords':
                col = col.split(', ')
            current_row[headers[j]] = col
            
    print(csv_dict[list(csv_dict.keys())[0]])
    print(csv_dict.keys())

{'Name': '6.7FRhcrtR-Gal4-uasKaede', 'Identifier (Name)': 'a150Tg ; Tg(UAS:Kaede)\t\n', 'Identifier (Link)': 'http://zfin.org/action/feature/view/ZDB-ALT-151028-3 ; http://zfin.org/ZDB-TGCONSTRCT-070117-81', 'Keywords': ['Tg(-6.7Tru.Hcrt:GAL4-VP16)', 'Tg(Uas:Kaede)', 'Hypocretin receptor', 'Vestibular', 'Gal4', 'UAS', 'Kaede'], 'Regions of Expression': '', 'Number of Fish Imaged': '12', 'Source (Reagent)': 'Schier Lab, Engert Lab, Baier Lab', 'Source (Images)': 'Schier, Engert', 'Notes': '5KB upstream region of the hypocretin receptor driving Gal4 expression. Labels neurons in the vestibular nucleus important for the vestibulo-ocular reflex', 'References': 'Randlett et al., Nature Methods, 2015, doi:10.1038/nmeth.3581 , Schoppik et al., J.Neurosci, 2017; DOI: https://doi.org/10.1523/JNEUROSCI.1711-17.2017'}
dict_keys(['6.7FRhcrtR-Gal4-uasKaede', 'Anti-5HT', 'Anti-Gad67', 'Anti-GlyR', 'Anti-TH', 'Anti-Zn12(Hnk-1)', 'Anti-Zn1', 'Anti-Znp1(Synaptotagmin2)', 'Anti-Zrf1(GFAP)', 'Anti-Zrf2',

#### Create PostgreSQL table for annotations, transfer csv content

In [77]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

create_table_command = (
"""
    DROP TABLE IF EXISTS annotations;
    CREATE TABLE annotations (
        line_id SERIAL PRIMARY KEY,
        line_name TEXT NOT NULL,
        identifier_name TEXT,
        identifier_link TEXT,
        keywords TEXT [],
        notes TEXT,
        refs TEXT,
        regions_of_expression TEXT,
        number_of_fish_imaged INT,
        source_reagent TEXT,
        source_images TEXT
    );
""")

In [78]:
try:
    cursor.execute(create_table_command)
    cursor.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    conn.close()

In [79]:
def list_to_pg2_array(lst):
    if lst:
        formatted_lst = str(lst).replace('[', '{').replace(']', '}').replace('\'', '\"')
    else:
        formatted_lst = "{}"
    return formatted_lst

In [82]:
def create_insert_commands(csv_dict):
    for i, key in enumerate(csv_dict.keys()):
        line = csv_dict[key]
        insert_command = (
            """
            INSERT INTO annotations
                (line_id, line_name, identifier_name, identifier_link,
                keywords, notes, regions_of_expression, number_of_fish_imaged,
                source_reagent, source_images, refs) 
                values (%i, '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s') 
            """ % (
                i, key, line['Identifier (Name)'], line['Identifier (Link)'],
                list_to_pg2_array(line['Keywords']), line['Notes'], 
                line['Regions of Expression'], line['Number of Fish Imaged'], 
                line['Source (Reagent)'], line['Source (Images)'], line['References']
            ))
        yield insert_command

#### Insert to table

In [83]:
insert_commands = list(create_insert_commands(csv_dict))

In [84]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

for insert_command in insert_commands:
    try:
        cursor.execute(insert_command)
    except Exception as e:
        print(e)
        
cursor.close()
conn.commit()

#### Check completed table

In [85]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

cursor.execute('SELECT * FROM annotations order by line_id')
rows = cursor.fetchall()

for row in rows[:]:
    print(row)

(0, '6.7FRhcrtR-Gal4-uasKaede', 'a150Tg ; Tg(UAS:Kaede)\t\n', 'http://zfin.org/action/feature/view/ZDB-ALT-151028-3 ; http://zfin.org/ZDB-TGCONSTRCT-070117-81', ['Tg(-6.7Tru.Hcrt:GAL4-VP16)', 'Tg(Uas:Kaede)', 'Hypocretin receptor', 'Vestibular', 'Gal4', 'UAS', 'Kaede'], '5KB upstream region of the hypocretin receptor driving Gal4 expression. Labels neurons in the vestibular nucleus important for the vestibulo-ocular reflex', 'Randlett et al., Nature Methods, 2015, doi:10.1038/nmeth.3581 , Schoppik et al., J.Neurosci, 2017; DOI: https://doi.org/10.1523/JNEUROSCI.1711-17.2017', '', 12, 'Schier Lab, Engert Lab, Baier Lab', 'Schier, Engert')
(1, 'Anti-5HT', 'Ab1-serotonin', 'http://zfin.org/ZDB-ATB-081017-7', ['Serotonin', '5HT', 'antibody'], 'Used at 1:100 AB concentration, Staining quality is relatively poor, with lots of background', 'Randlett et al., Nature Methods, 2015, doi:10.1038/nmeth.3581', '', 40, 'Sigma', 'Schier, Engert')
(2, 'Anti-Gad67', 'Ab1-gad', 'http://zfin.org/ZDB-ATB-0